In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
from datetime import datetime
from dateutil import parser

import uuid

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from datetime import datetime
from dateutil import parser
import time
from urllib import request
import csv
import re


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [3]:
# Pour DL Chromedriver  : # Pour DL Chromedriver  : https://sites.google.com/a/chromium.org/chromedriver/

In [4]:
attributs_labels = []
games_list = []
attributs_labels_players = []
teams_stats_list = []
players_stats_list = []
link_pages = []

max_players = 1000000000
# Environ pour 20 ans(?)
max_games = 30000
# max_games = 2000

In [5]:
def SaveInCsv(data):
    games = []
  
    for item in data:    
        games.append({
            'game_id': item.get('game_id'),
            'date_game': item.get('date_game'),
            'game_start_time': item.get('game_start_time'),
            'v_team_name': item.get('v_team_name'),
            'v_pts': item.get('v_pts'),
            'v_record': item.get('v_record'),
            'v_field_goals': item.get('v_field_goals'),
            'v_field_goal_attempts': item.get('v_field_goal_attempts'),
            'v_field_goal_percentage': item.get('v_field_goal_percentage'),
            'v_three_points_field_goals': item.get('v_three_points_field_goals'),
            'v_three_points_field_goal_attempts': item.get('v_three_points_field_goal_attempts'),
            'v_three_points_field_goal_percentage': item.get('v_three_points_field_goal_percentage'),
            'v_free_throws': item.get('v_free_throws'),
            'v_free_throw_attempts': item.get('v_free_throw_attempts'),
            'v_free_throw_percentage': item.get('v_free_throw_percentage'),
            'v_offensive_rebounds': item.get('v_offensive_rebounds'),
            'v_defensive_rebounds': item.get('v_defensive_rebounds'),
            'v_total_rebounds': item.get('v_total_rebounds'),
            'v_assists': item.get('v_assists'),
            'v_steals': item.get('v_steals'),
            'v_blocks': item.get('v_blocks'),
            'v_turn_over': item.get('v_turn_over'),
            'v_personnal_fouls': item.get('v_personnal_fouls'),
            'h_team_name': item.get('h_team_name'),
            'h_pts': item.get('h_pts'),
            'h_record': item.get('h_record'),
            'h_field_goals': item.get('h_field_goals'),
            'h_field_goal_attempts': item.get('h_field_goal_attempts'),
            'h_field_goal_percentage': item.get('h_field_goal_percentage'),
            'h_three_points_field_goals': item.get('h_three_points_field_goals'),
            'h_three_points_field_goal_attempts': item.get('h_three_points_field_goal_attempts'),
            'h_three_points_field_goal_percentage': item.get('h_three_points_field_goal_percentage'),
            'h_free_throws': item.get('h_free_throws'),
            'h_free_throw_attempts': item.get('h_free_throw_attempts'),
            'h_free_throw_percentage': item.get('h_free_throw_percentage'),
            'h_offensive_rebounds': item.get('h_offensive_rebounds'),
            'h_defensive_rebounds': item.get('h_defensive_rebounds'),
            'h_total_rebounds': item.get('h_total_rebounds'),
            'h_assists': item.get('h_assists'),
            'h_steals': item.get('h_steals'),
            'h_blocks': item.get('h_blocks'),
            'h_turn_over': item.get('h_turn_over'),
            'h_personnal_fouls': item.get('h_personnal_fouls'),
            'overtimes': item.get('overtimes'),
            'attendance': item.get('attendance'),
            'game_remarks': item.get('game_remarks')
        })

    keys = games[0].keys()
    with open('nba_games_data.csv', 'w', newline='', encoding='utf-8')  as output_file:
        dict_writer = csv.DictWriter(output_file, keys, delimiter=';')
        dict_writer.writeheader()
        dict_writer.writerows(games)

In [6]:
def SaveInCsv_players(data):
    players = []
  
    for item in data:    
        players.append({
            'game_id':  item.get('game_id'),
            'player_name': item.get('player_name'),
            'minutes_played': item.get('minutes_played'),
            'field_goals': item.get('field_goals'),
            'field_goal_attempts': item.get('field_goal_attempts'),
            'field_goal_percentage': item.get('field_goal_percentage'),
            'three_points_field_goals': item.get('three_points_field_goals'),
            'three_points_field_goal_attempts': item.get('three_points_field_goal_attempts'),
            'three_points_field_goal_percentage': item.get('three_points_field_goal_percentage'),
            'free_throws': item.get('free_throws'),
            'free_throw_attempts': item.get('free_throw_attempts'),
            'free_throw_percentage': item.get('free_throw_percentage'),
            'offensive_rebounds': item.get('offensive_rebounds'),
            'defensive_rebounds': item.get('defensive_rebounds'),
            'total_rebounds': item.get('total_rebounds'),
            'assists': item.get('assists'),
            'steals': item.get('steals'),
            'blocks': item.get('blocks'),
            'turn_over': item.get('turn_over'),
            'personnal_fouls': item.get('personnal_fouls'),
            'points': item.get('points'),
            'plus_minus': item.get('plus_minus'),
        })

    keys = players[0].keys()
    with open('nba_player_stats_data.csv', 'w', newline='', encoding='UTF-8')  as output_file:
        dict_writer = csv.DictWriter(output_file, keys, delimiter=';')
        dict_writer.writeheader()
        dict_writer.writerows(players)


#### Fonction pour récupérer les stats générales des games

In [7]:
def scrap_data(scrap_team_nb, scrap_player_nb):
    
#     browser.get("https://www.basketball-reference.com/"+link) 
#     html = browser.page_source

    soup = BeautifulSoup(html, 'lxml')
    schedule = soup.find(class_='table_container is_setup')

    schedule_sorted = soup.find(class_='table_container is_setup')
    tbody = schedule_sorted.find('tbody')

    # On cherche et retire les ligne _thead
    unwanted_thead = tbody.find_all(class_='thead')
    for unwanted in unwanted_thead:
        unwanted.extract()

    for row in tbody.find_all('tr'):
#         On décommente si on veut cut juste après le décompte - On laisse commenter pour finir l'année
#         if (max_games <= len(games_list)):
#             is_prev = False
#             break;
        
        current_url = browser.current_url
        
        game_id = str(uuid.uuid4())
        
        match_stats = [] 
        

        try:
            game_box_score = row.find(attrs={"data-stat":'box_score_text'})

            link = game_box_score.find('a', href=True)
#     On recupére le nombre de joueurs, le record visiteur et home dans data
            data = navigate_on_box_score("https://www.basketball-reference.com/"+link['href'], current_url, game_id, scrap_player_nb)
            scrap_player_nb = data[0]
        
            visitor_record = data[1]
            stats_visitor = data[2]
            
            home_record = data[3]
            stats_home = data[4]
            
        except:
            print("Pas de données disponibles")
            
            visitor_record = ""
            stats_visitor = {
                'field_goals': "",
                'field_goal_attempts': "",
                'field_goal_percentage': "",
                'three_points_field_goals': "",
                'three_points_field_goal_attempts': "",
                'three_points_field_goal_percentage': "",
                'free_throws': "",
                'free_throw_attempts': "",
                'free_throw_percentage': "",
                'offensive_rebounds': "",
                'defensive_rebounds': "",
                'total_rebounds': "",
                'assists': "",
                'steals': "",
                'blocks': "",
                'turn_over': "",
                'personnal_fouls': ""
            }
            
            home_record = ""
            stats_home = {
                'field_goals': "",
                'field_goal_attempts': "",
                'field_goal_percentage': "",
                'three_points_field_goals': "",
                'three_points_field_goal_attempts': "",
                'three_points_field_goal_percentage': "",
                'free_throws': "",
                'free_throw_attempts': "",
                'free_throw_percentage': "",
                'offensive_rebounds': "",
                'defensive_rebounds': "",
                'total_rebounds': "",
                'assists': "",
                'steals': "",
                'blocks': "",
                'turn_over': "",
                'personnal_fouls': ""
            }
    
#         Liste des stats dans le tableau
        stats_match = ['date_game', 'game_start_time', 'visitor_team_name','visitor_pts', 
                 'home_team_name', 'home_pts', 'box_score_text', 
                 'overtimes', 'attendance', 'game_remarks']
    
        
        for stat in stats_match:
            try:
                match_data = row.find(attrs={"data-stat":stat}).text
                match_stats.append(match_data)
            except:
                match_stats.append("")
        
        new_game = {
            'game_id': game_id,
            'date_game': match_stats[0], 
            'game_start_time': match_stats[1],
            'v_team_name': match_stats[2],
            'v_pts': match_stats[3],
            'v_record': visitor_record,
            'v_field_goals': stats_visitor['field_goals'],
            'v_field_goal_attempts': stats_visitor['field_goal_attempts'],
            'v_field_goal_percentage': stats_visitor['field_goal_percentage'],
            'v_three_points_field_goals': stats_visitor['three_points_field_goals'],
            'v_three_points_field_goal_attempts': stats_visitor['three_points_field_goal_attempts'],
            'v_three_points_field_goal_percentage': stats_visitor['three_points_field_goal_percentage'],
            'v_free_throws': stats_visitor['free_throws'],
            'v_free_throw_attempts': stats_visitor['free_throw_attempts'],
            'v_free_throw_percentage': stats_visitor['free_throw_percentage'],
            'v_offensive_rebounds': stats_visitor['offensive_rebounds'],
            'v_defensive_rebounds': stats_visitor['defensive_rebounds'],
            'v_total_rebounds': stats_visitor['total_rebounds'],
            'v_assists': stats_visitor['assists'],
            'v_steals': stats_visitor['steals'],
            'v_blocks': stats_visitor['blocks'],
            'v_turn_over': stats_visitor['turn_over'],
            'v_personnal_fouls': stats_visitor['personnal_fouls'],
            'h_team_name': match_stats[4],
            'h_pts': match_stats[5],
            'h_record': home_record,
            'h_field_goals': stats_home['field_goals'],
            'h_field_goal_attempts': stats_home['field_goal_attempts'],
            'h_field_goal_percentage': stats_home['field_goal_percentage'],
            'h_three_points_field_goals': stats_home['three_points_field_goals'],
            'h_three_points_field_goal_attempts': stats_home['three_points_field_goal_attempts'],
            'h_three_points_field_goal_percentage': stats_home['three_points_field_goal_percentage'],
            'h_free_throws': stats_home['free_throws'],
            'h_free_throw_attempts': stats_home['free_throw_attempts'],
            'h_free_throw_percentage': stats_home['free_throw_percentage'],
            'h_offensive_rebounds': stats_home['offensive_rebounds'],
            'h_defensive_rebounds': stats_home['defensive_rebounds'],
            'h_total_rebounds': stats_home['total_rebounds'],
            'h_assists': stats_home['assists'],
            'h_steals': stats_home['steals'],
            'h_blocks': stats_home['blocks'],
            'h_turn_over': stats_home['turn_over'],
            'h_personnal_fouls': stats_home['personnal_fouls'],
            'box_score_text': match_stats[6],
            'overtimes': match_stats[7],
            'attendance': match_stats[8],
            'game_remarks': match_stats[9],
        }
        
        games_list.append(new_game)
        
        scrap_team_nb += 1
        print("Game #"+str(scrap_team_nb)+ " scrappée - "+str(scrap_player_nb)+" joueurs scrappés")
        
    return [scrap_team_nb, scrap_player_nb]

In [8]:
def navigate_on_box_score(url, url_back, game_id, scrap_player_nb):
    
    browser.get(url) 
    html = browser.page_source
    
#     On recupére le nb joueur - objet pour visiteur - objet home
    data = scrap_game_data(html, game_id, scrap_player_nb)
    
    scrap_player_nb = data[0]
    
    time.sleep(0.25)
    
    browser.get(url_back) 
    html = browser.page_source
    
    return data

#### Fonction pour récupérer les stats des joueurs dans la game

In [9]:
def scrap_game_data(html, game_id, scrap_player_nb):    
    soup = BeautifulSoup(html, 'lxml')
    
#     On récupère les records de team
    records = []
    teams_stats_list = []
    teams_record =  soup.findAll('div', id=re.compile("^box-...-game-basic_sh")) 
    
    for team_record in teams_record:
        try:
            record_data = team_record.find('h2').text
            
#             On récupère juste le score entre les parenthèses
            record_data_cleaned = record_data[record_data.find("(")+1:record_data.find(")")]
            records.append(record_data_cleaned)
        except:
            records.append("")
                
#     On récupère les stats des joueurs et des teams
    soup = BeautifulSoup(html, 'lxml')   
    stats_teams = soup.findAll('table', id=re.compile("^box-...-game-basic"))
    
    for team in stats_teams:
        
#         Liste des stats dans le tableau
        stats = ['player', 'mp', 'fg','fga', 
                 'fg_pct', 'fg3', 'fg3a', 
                 'fg3_pct', 'ft', 'fta', 'ft_pct',
                 'orb', 'drb', 'trb', 'ast', 'stl', 'blk',
                'tov', 'pf', 'pts', 'plus_minus']
        
        
#         On récupère les stats de l'équipe entière pour le match
        tfoot = team.find('tfoot')
        
        team_stats = []
        
        for stat in stats:
            try:
                team_data = tfoot.find(attrs={"data-stat":stat}).text
                team_stats.append(team_data)
            except:
                team_stats.append("")
                
  
        team_stats_item = {
            'field_goals': team_stats[2],
            'field_goal_attempts': team_stats[3],
            'field_goal_percentage': team_stats[4],
            'three_points_field_goals': team_stats[5],
            'three_points_field_goal_attempts': team_stats[6],
            'three_points_field_goal_percentage': team_stats[7],
            'free_throws': team_stats[8],
            'free_throw_attempts': team_stats[9],
            'free_throw_percentage': team_stats[10],
            'offensive_rebounds': team_stats[11],
            'defensive_rebounds': team_stats[12],
            'total_rebounds': team_stats[13],
            'assists': team_stats[14],
            'steals': team_stats[15],
            'blocks': team_stats[16],
            'turn_over': team_stats[17],
            'personnal_fouls': team_stats[18],
        }
        
        teams_stats_list.append(team_stats_item)
        
#         On récupère les stats de chaque jour pour le match
        tbody = team.find('tbody')
        
#         On cherche et retire les ligne _thead
        unwanted_thead = tbody.find_all(class_='thead')
        for unwanted in unwanted_thead:
            unwanted.extract()
        
        for row in tbody.find_all('tr') :

            player_stats = []            
            
            for stat in stats:
                try:
                    data = row.find(attrs={"data-stat":stat}).text
                    player_stats.append(data)
                except:
                    player_stats.append("")
                
  
            player_stats_item = {
                'game_id': game_id,
                'player_name': player_stats[0],
                'minutes_played': player_stats[1],
                'field_goals': player_stats[2],
                'field_goal_attempts': player_stats[3],
                'field_goal_percentage': player_stats[4],
                'three_points_field_goals': player_stats[5],
                'three_points_field_goal_attempts': player_stats[6],
                'three_points_field_goal_percentage': player_stats[7],
                'free_throws': player_stats[8],
                'free_throw_attempts': player_stats[9],
                'free_throw_percentage': player_stats[10],
                'offensive_rebounds': player_stats[11],
                'defensive_rebounds': player_stats[12],
                'total_rebounds': player_stats[13],
                'assists': player_stats[14],
                'steals': player_stats[15],
                'blocks': player_stats[16],
                'turn_over': player_stats[17],
                'personnal_fouls': player_stats[18],
                'points': player_stats[19],
                'plus_minus': player_stats[20],
            }

            players_stats_list.append(player_stats_item)

            scrap_player_nb += 1

#     data = [scrap_player_nb, home_record, visitor_home]
#     data = [scrap_player_nb, records[0], records[1]]
    
#     On va envoyer le nb joueur - objet stats team visitor - objet stats team home
    data = [scrap_player_nb, records[0], teams_stats_list[0], records[1], teams_stats_list[1]]
    
    return data

In [10]:
# scrap_game_data('https://www.basketball-reference.com/boxscores/201810160BOS.html')

In [11]:
html = None

# url = 'https://www.basketball-reference.com/leagues/NBA_2019_games.html'

url = 'https://www.basketball-reference.com/leagues/NBA_2021_games.html'


chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-plugins-discovery")
chrome_options.add_argument("--start-maximized")

browser = webdriver.Chrome('C:/Users/beren/Desktop/chromedriver.exe', options=chrome_options)


browser.get(url)
html = browser.page_source

time.sleep(1)

# aria-label="AGREE"
# soup.find('button', id=re.compile("^box-...-game-basic"))

# Changer la class a chaque fois
cookie_button = browser.find_element_by_class_name("css-47sehv")
cookie_button.click()

soup = BeautifulSoup(html, 'lxml')

is_prev = True
scrap_team_nb = 0
scrap_player_nb = 0

while is_prev == True:

    # On récupère les liens pour pouvoir naviguer de mois en mois
    link_pages = []
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    filter = soup.find(class_='filter')
    # print(filter)

    for link in filter.find_all('a'):
        link_pages.append(link.get('href'))
#     print(link_pages)

    for link in link_pages:
        
        browser.get("https://www.basketball-reference.com/"+link) 
        html = browser.page_source
        soup = BeautifulSoup(html, 'lxml')
        
#         On décommente si on veut finir à la fin du mois après avoir atteint la limite de nombre de match
#         if (max_games <= len(games_list)):
#             is_prev = False
#             break;
            
        results = scrap_data(scrap_team_nb, scrap_player_nb)
        
        scrap_team_nb = results[0]
        scrap_player_nb = results[1]
    
    time.sleep(0.5)
    
#     Permet de cut à la fin de l'année après le nombre de match max indiqué
    if (max_games <= len(games_list)):
        is_prev = False
        break;

    try:
        prev_button = browser.find_element_by_class_name("prev")
        prev_button.click()
    except:
        is_prev = False
    

# browser.quit()


Game #1 scrappée - 28 joueurs scrappés
Game #2 scrappée - 56 joueurs scrappés
Game #3 scrappée - 83 joueurs scrappés
Game #4 scrappée - 112 joueurs scrappés
Game #5 scrappée - 141 joueurs scrappés
Game #6 scrappée - 170 joueurs scrappés
Game #7 scrappée - 199 joueurs scrappés
Game #8 scrappée - 228 joueurs scrappés
Game #9 scrappée - 255 joueurs scrappés
Game #10 scrappée - 281 joueurs scrappés
Game #11 scrappée - 309 joueurs scrappés
Game #12 scrappée - 338 joueurs scrappés
Game #13 scrappée - 364 joueurs scrappés
Game #14 scrappée - 392 joueurs scrappés
Game #15 scrappée - 421 joueurs scrappés
Game #16 scrappée - 449 joueurs scrappés
Game #17 scrappée - 477 joueurs scrappés
Game #18 scrappée - 505 joueurs scrappés
Game #19 scrappée - 534 joueurs scrappés
Game #20 scrappée - 559 joueurs scrappés
Game #21 scrappée - 587 joueurs scrappés
Game #22 scrappée - 616 joueurs scrappés
Game #23 scrappée - 644 joueurs scrappés
Game #24 scrappée - 672 joueurs scrappés
Game #25 scrappée - 700 joue

Game #195 scrappée - 5269 joueurs scrappés
Game #196 scrappée - 5294 joueurs scrappés
Game #197 scrappée - 5324 joueurs scrappés
Game #198 scrappée - 5349 joueurs scrappés
Game #199 scrappée - 5378 joueurs scrappés
Game #200 scrappée - 5406 joueurs scrappés
Game #201 scrappée - 5430 joueurs scrappés
Game #202 scrappée - 5457 joueurs scrappés
Game #203 scrappée - 5484 joueurs scrappés
Game #204 scrappée - 5513 joueurs scrappés
Game #205 scrappée - 5543 joueurs scrappés
Game #206 scrappée - 5568 joueurs scrappés
Game #207 scrappée - 5594 joueurs scrappés
Game #208 scrappée - 5618 joueurs scrappés
Game #209 scrappée - 5646 joueurs scrappés
Game #210 scrappée - 5673 joueurs scrappés
Game #211 scrappée - 5701 joueurs scrappés
Game #212 scrappée - 5729 joueurs scrappés
Game #213 scrappée - 5757 joueurs scrappés
Game #214 scrappée - 5784 joueurs scrappés
Game #215 scrappée - 5811 joueurs scrappés
Game #216 scrappée - 5836 joueurs scrappés
Game #217 scrappée - 5863 joueurs scrappés
Game #218 s

Game #386 scrappée - 10315 joueurs scrappés
Game #387 scrappée - 10340 joueurs scrappés
Game #388 scrappée - 10364 joueurs scrappés
Game #389 scrappée - 10392 joueurs scrappés
Game #390 scrappée - 10420 joueurs scrappés
Game #391 scrappée - 10445 joueurs scrappés
Game #392 scrappée - 10471 joueurs scrappés
Game #393 scrappée - 10496 joueurs scrappés
Game #394 scrappée - 10524 joueurs scrappés
Game #395 scrappée - 10548 joueurs scrappés
Game #396 scrappée - 10574 joueurs scrappés
Game #397 scrappée - 10601 joueurs scrappés
Game #398 scrappée - 10625 joueurs scrappés
Game #399 scrappée - 10652 joueurs scrappés
Game #400 scrappée - 10676 joueurs scrappés
Game #401 scrappée - 10702 joueurs scrappés
Game #402 scrappée - 10727 joueurs scrappés
Game #403 scrappée - 10753 joueurs scrappés
Game #404 scrappée - 10777 joueurs scrappés
Game #405 scrappée - 10804 joueurs scrappés
Game #406 scrappée - 10830 joueurs scrappés
Game #407 scrappée - 10856 joueurs scrappés
Game #408 scrappée - 10882 joueu

Game #573 scrappée - 15202 joueurs scrappés
Game #574 scrappée - 15227 joueurs scrappés
Game #575 scrappée - 15255 joueurs scrappés
Game #576 scrappée - 15284 joueurs scrappés
Game #577 scrappée - 15313 joueurs scrappés
Game #578 scrappée - 15340 joueurs scrappés
Game #579 scrappée - 15370 joueurs scrappés
Game #580 scrappée - 15397 joueurs scrappés
Game #581 scrappée - 15421 joueurs scrappés
Game #582 scrappée - 15449 joueurs scrappés
Game #583 scrappée - 15477 joueurs scrappés
Game #584 scrappée - 15505 joueurs scrappés
Game #585 scrappée - 15529 joueurs scrappés
Game #586 scrappée - 15556 joueurs scrappés
Game #587 scrappée - 15583 joueurs scrappés
Game #588 scrappée - 15613 joueurs scrappés
Game #589 scrappée - 15642 joueurs scrappés
Game #590 scrappée - 15670 joueurs scrappés
Game #591 scrappée - 15697 joueurs scrappés
Game #592 scrappée - 15722 joueurs scrappés
Game #593 scrappée - 15751 joueurs scrappés
Game #594 scrappée - 15781 joueurs scrappés
Game #595 scrappée - 15808 joueu

Game #760 scrappée - 20255 joueurs scrappés
Game #761 scrappée - 20284 joueurs scrappés
Game #762 scrappée - 20312 joueurs scrappés
Game #763 scrappée - 20338 joueurs scrappés
Game #764 scrappée - 20362 joueurs scrappés
Game #765 scrappée - 20389 joueurs scrappés
Game #766 scrappée - 20418 joueurs scrappés
Game #767 scrappée - 20447 joueurs scrappés
Game #768 scrappée - 20474 joueurs scrappés
Game #769 scrappée - 20501 joueurs scrappés
Game #770 scrappée - 20529 joueurs scrappés
Game #771 scrappée - 20556 joueurs scrappés
Game #772 scrappée - 20583 joueurs scrappés
Game #773 scrappée - 20610 joueurs scrappés
Game #774 scrappée - 20637 joueurs scrappés
Game #775 scrappée - 20665 joueurs scrappés
Game #776 scrappée - 20692 joueurs scrappés
Game #777 scrappée - 20717 joueurs scrappés
Game #778 scrappée - 20742 joueurs scrappés
Game #779 scrappée - 20768 joueurs scrappés
Game #780 scrappée - 20795 joueurs scrappés
Game #781 scrappée - 20825 joueurs scrappés
Game #782 scrappée - 20851 joueu

...

Game #1130 scrappée - 30117 joueurs scrappés
Game #1131 scrappée - 30143 joueurs scrappés
Game #1132 scrappée - 30168 joueurs scrappés
Game #1133 scrappée - 30192 joueurs scrappés
Game #1134 scrappée - 30216 joueurs scrappés
Game #1135 scrappée - 30242 joueurs scrappés
Game #1136 scrappée - 30268 joueurs scrappés
Game #1137 scrappée - 30294 joueurs scrappés
Game #1138 scrappée - 30319 joueurs scrappés
Game #1139 scrappée - 30345 joueurs scrappés
Game #1140 scrappée - 30370 joueurs scrappés
Game #1141 scrappée - 30395 joueurs scrappés
Game #1142 scrappée - 30420 joueurs scrappés
Game #1143 scrappée - 30445 joueurs scrappés
Game #1144 scrappée - 30471 joueurs scrappés
Game #1145 scrappée - 30497 joueurs scrappés
Game #1146 scrappée - 30522 joueurs scrappés
Game #1147 scrappée - 30547 joueurs scrappés
Game #1148 scrappée - 30571 joueurs scrappés
Game #1149 scrappée - 30597 joueurs scrappés
Game #1150 scrappée - 30623 joueurs scrappés
Game #1151 scrappée - 30648 joueurs scrappés
Game #1152

Game #1313 scrappée - 34713 joueurs scrappés
Game #1314 scrappée - 34736 joueurs scrappés
Game #1315 scrappée - 34762 joueurs scrappés
Game #1316 scrappée - 34786 joueurs scrappés
Game #1317 scrappée - 34811 joueurs scrappés
Game #1318 scrappée - 34836 joueurs scrappés
Game #1319 scrappée - 34861 joueurs scrappés
Game #1320 scrappée - 34886 joueurs scrappés
Game #1321 scrappée - 34911 joueurs scrappés
Game #1322 scrappée - 34936 joueurs scrappés
Game #1323 scrappée - 34962 joueurs scrappés
Game #1324 scrappée - 34987 joueurs scrappés
Game #1325 scrappée - 35011 joueurs scrappés
Game #1326 scrappée - 35036 joueurs scrappés
Game #1327 scrappée - 35060 joueurs scrappés
Game #1328 scrappée - 35085 joueurs scrappés
Game #1329 scrappée - 35111 joueurs scrappés
Game #1330 scrappée - 35136 joueurs scrappés
Game #1331 scrappée - 35161 joueurs scrappés
Game #1332 scrappée - 35185 joueurs scrappés
Game #1333 scrappée - 35210 joueurs scrappés
Game #1334 scrappée - 35235 joueurs scrappés
Game #1335

Game #1496 scrappée - 39283 joueurs scrappés
Game #1497 scrappée - 39308 joueurs scrappés
Game #1498 scrappée - 39331 joueurs scrappés
Game #1499 scrappée - 39357 joueurs scrappés
Game #1500 scrappée - 39381 joueurs scrappés
Game #1501 scrappée - 39405 joueurs scrappés
Game #1502 scrappée - 39431 joueurs scrappés
Game #1503 scrappée - 39457 joueurs scrappés
Game #1504 scrappée - 39482 joueurs scrappés
Game #1505 scrappée - 39506 joueurs scrappés
Game #1506 scrappée - 39532 joueurs scrappés
Game #1507 scrappée - 39555 joueurs scrappés
Game #1508 scrappée - 39577 joueurs scrappés
Game #1509 scrappée - 39601 joueurs scrappés
Game #1510 scrappée - 39625 joueurs scrappés
Game #1511 scrappée - 39650 joueurs scrappés
Game #1512 scrappée - 39676 joueurs scrappés
Game #1513 scrappée - 39701 joueurs scrappés
Game #1514 scrappée - 39726 joueurs scrappés
Game #1515 scrappée - 39751 joueurs scrappés
Game #1516 scrappée - 39776 joueurs scrappés
Game #1517 scrappée - 39799 joueurs scrappés
Game #1518

Game #1679 scrappée - 43853 joueurs scrappés
Game #1680 scrappée - 43877 joueurs scrappés
Game #1681 scrappée - 43902 joueurs scrappés
Game #1682 scrappée - 43927 joueurs scrappés
Game #1683 scrappée - 43951 joueurs scrappés
Game #1684 scrappée - 43976 joueurs scrappés
Game #1685 scrappée - 44001 joueurs scrappés
Game #1686 scrappée - 44026 joueurs scrappés
Game #1687 scrappée - 44051 joueurs scrappés
Game #1688 scrappée - 44075 joueurs scrappés
Game #1689 scrappée - 44100 joueurs scrappés
Game #1690 scrappée - 44124 joueurs scrappés
Game #1691 scrappée - 44150 joueurs scrappés
Game #1692 scrappée - 44176 joueurs scrappés
Game #1693 scrappée - 44201 joueurs scrappés
Game #1694 scrappée - 44226 joueurs scrappés
Game #1695 scrappée - 44251 joueurs scrappés
Game #1696 scrappée - 44276 joueurs scrappés
Game #1697 scrappée - 44300 joueurs scrappés
Game #1698 scrappée - 44326 joueurs scrappés
Game #1699 scrappée - 44349 joueurs scrappés
Game #1700 scrappée - 44375 joueurs scrappés
Game #1701

Game #1862 scrappée - 48419 joueurs scrappés
Game #1863 scrappée - 48443 joueurs scrappés
Game #1864 scrappée - 48469 joueurs scrappés
Game #1865 scrappée - 48495 joueurs scrappés
Game #1866 scrappée - 48521 joueurs scrappés
Game #1867 scrappée - 48547 joueurs scrappés
Game #1868 scrappée - 48572 joueurs scrappés
Game #1869 scrappée - 48597 joueurs scrappés
Game #1870 scrappée - 48622 joueurs scrappés
Game #1871 scrappée - 48648 joueurs scrappés
Game #1872 scrappée - 48672 joueurs scrappés
Game #1873 scrappée - 48696 joueurs scrappés
Game #1874 scrappée - 48722 joueurs scrappés
Game #1875 scrappée - 48748 joueurs scrappés
Game #1876 scrappée - 48774 joueurs scrappés
Game #1877 scrappée - 48799 joueurs scrappés
Game #1878 scrappée - 48823 joueurs scrappés
Game #1879 scrappée - 48848 joueurs scrappés
Game #1880 scrappée - 48874 joueurs scrappés
Game #1881 scrappée - 48900 joueurs scrappés
Game #1882 scrappée - 48925 joueurs scrappés
Game #1883 scrappée - 48950 joueurs scrappés
Game #1884

In [12]:
SaveInCsv(games_list)
SaveInCsv_players(players_stats_list)